In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2061083,2022-01-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2061084,2022-01-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2061085,2022-01-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2061086,2022-01-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
53235,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
53237,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
53239,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
53241,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
53243,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1250733,2022-01-23,Arkansas,Arkansas,5001,5089,67.00,5000,Arkansas,AR,Arkansas,State,3017804
1250735,2022-01-24,Arkansas,Arkansas,5001,5116,67.00,5000,Arkansas,AR,Arkansas,State,3017804
1250737,2022-01-25,Arkansas,Arkansas,5001,5183,67.00,5000,Arkansas,AR,Arkansas,State,3017804
1250739,2022-01-26,Arkansas,Arkansas,5001,5237,67.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2061083,2022-01-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2061084,2022-01-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2061085,2022-01-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2061086,2022-01-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-01-27') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
737,2022-01-27,Snohomish,Washington,130585,923.00,53061,WA,County,822083,15884.65,112.28
1472,2022-01-27,Cook,Illinois,1071981,13745.00,17031,IL,County,5150233,20814.22,266.88
2206,2022-01-27,Orange,California,538115,6037.00,6059,CA,County,3175692,16944.81,190.10
2939,2022-01-27,Maricopa,Arizona,1150653,14585.00,4013,AZ,County,4485414,25653.22,325.17
3672,2022-01-27,Los Angeles,California,2587443,28715.00,6037,CA,County,10039107,25773.64,286.03
...,...,...,...,...,...,...,...,...,...,...,...
2059323,2022-01-27,Wheeler,Oregon,207,3.00,41069,OR,County,1332,15540.54,225.23
2059795,2022-01-27,King,Texas,39,0.00,48269,TX,County,272,14338.24,0.00
2060236,2022-01-27,Esmeralda,Nevada,85,2.00,32009,NV,County,873,9736.54,229.10
2060673,2022-01-27,Loving,Texas,99,0.00,48301,TX,County,169,58579.88,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
737,2022-01-27,Snohomish,Washington,130585,923.00,53061,WA,County,822083,15884.65,112.28,112.28,15884.65
1472,2022-01-27,Cook,Illinois,1071981,13745.00,17031,IL,County,5150233,20814.22,266.88,266.88,20814.22
2206,2022-01-27,Orange,California,538115,6037.00,6059,CA,County,3175692,16944.81,190.10,190.10,16944.81
2939,2022-01-27,Maricopa,Arizona,1150653,14585.00,4013,AZ,County,4485414,25653.22,325.17,325.17,25653.22
3672,2022-01-27,Los Angeles,California,2587443,28715.00,6037,CA,County,10039107,25773.64,286.03,286.03,25773.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059323,2022-01-27,Wheeler,Oregon,207,3.00,41069,OR,County,1332,15540.54,225.23,225.23,15540.54
2059795,2022-01-27,King,Texas,39,0.00,48269,TX,County,272,14338.24,0.00,0.00,14338.24
2060236,2022-01-27,Esmeralda,Nevada,85,2.00,32009,NV,County,873,9736.54,229.10,229.10,9736.54
2060673,2022-01-27,Loving,Texas,99,0.00,48301,TX,County,169,58579.88,0.00,0.00,58579.88


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1308358,2022-01-27,Hale,Alabama,4226,94.00,1065,AL,County,14651,28844.45,641.59,641.59,28844.45,1
902289,2022-01-27,Clay,Alabama,3620,71.00,1027,AL,County,13235,27351.72,536.46,536.46,27351.72,2
758436,2022-01-27,Franklin,Alabama,8565,116.00,1059,AL,County,31362,27310.12,369.87,369.87,27310.12,3
1425353,2022-01-27,Clarke,Alabama,6415,89.00,1025,AL,County,23622,27156.89,376.77,376.77,27156.89,4
1103682,2022-01-27,Winston,Alabama,6323,114.00,1133,AL,County,23629,26759.49,482.46,482.46,26759.49,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954395,2022-01-27,Weston,Wyoming,1426,16.00,56045,WY,County,6927,20586.11,230.98,230.98,20586.11,19
1765956,2022-01-27,Big Horn,Wyoming,2374,60.00,56003,WY,County,11790,20135.71,508.91,508.91,20135.71,20
1555225,2022-01-27,Lincoln,Wyoming,3710,27.00,56023,WY,County,19830,18709.03,136.16,136.16,18709.03,21
1248731,2022-01-27,Sublette,Wyoming,1767,25.00,56035,WY,County,9831,17973.76,254.30,254.30,17973.76,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1021581,2022-01-27,Lowndes,Alabama,2375,68.00,1085,AL,County,9726,24419.08,699.16,699.16,24419.08,25,1
1308358,2022-01-27,Hale,Alabama,4226,94.00,1065,AL,County,14651,28844.45,641.59,641.59,28844.45,1,2
432038,2022-01-27,Walker,Alabama,15805,392.00,1127,AL,County,63521,24881.54,617.12,617.12,24881.54,21,3
1020235,2022-01-27,Crenshaw,Alabama,3383,81.00,1041,AL,County,13772,24564.33,588.15,588.15,24564.33,22,4
904311,2022-01-27,Dallas,Alabama,7314,218.00,1047,AL,County,37196,19663.40,586.08,586.08,19663.40,62,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140117,2022-01-27,Sheridan,Wyoming,7430,63.00,56033,WY,County,30485,24372.64,206.66,206.66,24372.64,8,19
1515990,2022-01-27,Uinta,Wyoming,5222,34.00,56041,WY,County,20226,25818.25,168.10,168.10,25818.25,5,20
1555225,2022-01-27,Lincoln,Wyoming,3710,27.00,56023,WY,County,19830,18709.03,136.16,136.16,18709.03,21,21
1017543,2022-01-27,Albany,Wyoming,9226,45.00,56001,WY,County,38880,23729.42,115.74,115.74,23729.42,11,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,112.28,15884.65,30,19
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,112.28,15884.65,30,19
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,112.28,15884.65,30,19
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,112.28,15884.65,30,19
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,112.28,15884.65,30,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056323,2022-01-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2056324,2022-01-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2056325,2022-01-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2056326,2022-01-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1302930,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,641.59,28844.45,2,1,1.00,nan
1302931,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,641.59,28844.45,2,1,0.00,nan
1302932,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,641.59,28844.45,2,1,0.00,nan
1302933,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,641.59,28844.45,2,1,0.00,nan
1302934,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,641.59,28844.45,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581669,2022-01-23,Crook,Wyoming,1154,22.00,56011,WY,County,7584,15216.24,290.08,290.08,16046.94,12,23,0.00,7.71
1581670,2022-01-24,Crook,Wyoming,1171,22.00,56011,WY,County,7584,15440.40,290.08,290.08,16046.94,12,23,17.00,8.00
1581671,2022-01-25,Crook,Wyoming,1190,22.00,56011,WY,County,7584,15690.93,290.08,290.08,16046.94,12,23,19.00,9.00
1581672,2022-01-26,Crook,Wyoming,1206,22.00,56011,WY,County,7584,15901.90,290.08,290.08,16046.94,12,23,16.00,9.79


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1017491,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,24419.08,1,25,1.00,187.57,0.00,nan
1017492,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,24419.08,1,25,0.00,187.57,0.00,nan
1017493,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,24419.08,1,25,0.00,123.21,0.00,nan
1017494,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,24419.08,1,25,0.00,112.21,0.00,nan
1017495,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,24419.08,1,25,0.00,105.79,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427250,2022-01-23,Teton,Wyoming,8814,14.00,56039,WY,County,23464,37563.93,59.67,63.93,39187.69,23,1,0.00,127.79,0.00,0.00
427251,2022-01-24,Teton,Wyoming,8960,14.00,56039,WY,County,23464,38186.16,59.67,63.93,39187.69,23,1,146.00,117.71,0.00,0.00
427252,2022-01-25,Teton,Wyoming,9049,15.00,56039,WY,County,23464,38565.46,63.93,63.93,39187.69,23,1,89.00,108.29,1.00,0.07
427253,2022-01-26,Teton,Wyoming,9121,15.00,56039,WY,County,23464,38872.32,63.93,63.93,39187.69,23,1,72.00,104.50,0.00,0.07


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-01-27') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
516326,2022-01-27,Imperial,California,56552,838.00,6025,CA,County,181215,31207.13,462.43,462.43,31207.13,1,2,1323.00,638.50,7.00,1.79
1249331,2022-01-27,Tuolumne,California,11411,159.00,6109,CA,County,54478,20946.07,291.86,291.86,20946.07,2,15,138.00,153.21,1.00,0.50
256024,2022-01-27,San Bernardino,California,542214,6291.00,6071,CA,County,2180085,24871.23,288.57,288.57,24871.23,3,6,4425.00,5639.86,3.00,4.93
3672,2022-01-27,Los Angeles,California,2587443,28715.00,6037,CA,County,10039107,25773.64,286.03,286.03,25773.64,4,4,25971.00,32515.43,85.00,52.14
78816,2022-01-27,Shasta,California,32530,515.00,6089,CA,County,180080,18064.19,285.98,285.98,18064.19,5,27,769.00,284.71,0.00,0.93
113969,2022-01-27,Stanislaus,California,123084,1527.00,6099,CA,County,550660,22352.09,277.30,277.30,22352.09,6,12,1557.00,1270.36,1.00,1.71
907633,2022-01-27,Inyo,California,4193,50.00,6027,CA,County,18039,23244.08,277.18,277.18,23244.08,7,8,162.00,56.14,0.00,0.07
821925,2022-01-27,Merced,California,62415,740.00,6047,CA,County,277680,22477.31,266.49,266.49,22477.31,8,11,719.00,728.36,2.00,1.43
141486,2022-01-27,Tulare,California,116213,1234.00,6107,CA,County,466195,24927.98,264.70,264.70,24927.98,9,5,3086.00,1387.57,5.00,3.00
1102280,2022-01-27,Kings,California,47349,398.00,6031,CA,County,152940,30959.20,260.23,260.23,30959.20,10,3,881.00,579.93,2.00,0.50


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1940443,2022-01-27,Lassen,California,9558,61.00,6035,CA,County,30573,31262.88,199.52,199.52,31262.88,19,1,27.00,73.07,0.00,0.00
516326,2022-01-27,Imperial,California,56552,838.00,6025,CA,County,181215,31207.13,462.43,462.43,31207.13,1,2,1323.00,638.50,7.00,1.79
1102280,2022-01-27,Kings,California,47349,398.00,6031,CA,County,152940,30959.20,260.23,260.23,30959.20,10,3,881.00,579.93,2.00,0.50
3672,2022-01-27,Los Angeles,California,2587443,28715.00,6037,CA,County,10039107,25773.64,286.03,286.03,25773.64,4,4,25971.00,32515.43,85.00,52.14
141486,2022-01-27,Tulare,California,116213,1234.00,6107,CA,County,466195,24927.98,264.70,264.70,24927.98,9,5,3086.00,1387.57,5.00,3.00
256024,2022-01-27,San Bernardino,California,542214,6291.00,6071,CA,County,2180085,24871.23,288.57,288.57,24871.23,3,6,4425.00,5639.86,3.00,4.93
51850,2022-01-27,Madera,California,36680,336.00,6039,CA,County,157327,23314.50,213.57,213.57,23314.50,17,7,1451.00,491.86,7.00,0.50
907633,2022-01-27,Inyo,California,4193,50.00,6027,CA,County,18039,23244.08,277.18,277.18,23244.08,7,8,162.00,56.14,0.00,0.07
322317,2022-01-27,Kern,California,208859,2005.00,6029,CA,County,900202,23201.35,222.73,222.73,23201.35,15,9,1745.00,2124.93,17.00,3.36
52542,2022-01-27,Riverside,California,566883,5812.00,6065,CA,County,2470546,22945.66,235.25,235.25,22945.66,13,10,4601.00,6325.50,25.00,10.79


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
516326,2022-01-27,Imperial,California,56552,838.00,6025,CA,County,181215,31207.13,462.43,462.43,31207.13,1,2,1323.00,638.50,7.00,1.79
1249331,2022-01-27,Tuolumne,California,11411,159.00,6109,CA,County,54478,20946.07,291.86,291.86,20946.07,2,15,138.00,153.21,1.00,0.50
256024,2022-01-27,San Bernardino,California,542214,6291.00,6071,CA,County,2180085,24871.23,288.57,288.57,24871.23,3,6,4425.00,5639.86,3.00,4.93
3672,2022-01-27,Los Angeles,California,2587443,28715.00,6037,CA,County,10039107,25773.64,286.03,286.03,25773.64,4,4,25971.00,32515.43,85.00,52.14
78816,2022-01-27,Shasta,California,32530,515.00,6089,CA,County,180080,18064.19,285.98,285.98,18064.19,5,27,769.00,284.71,0.00,0.93
113969,2022-01-27,Stanislaus,California,123084,1527.00,6099,CA,County,550660,22352.09,277.30,277.30,22352.09,6,12,1557.00,1270.36,1.00,1.71
907633,2022-01-27,Inyo,California,4193,50.00,6027,CA,County,18039,23244.08,277.18,277.18,23244.08,7,8,162.00,56.14,0.00,0.07
821925,2022-01-27,Merced,California,62415,740.00,6047,CA,County,277680,22477.31,266.49,266.49,22477.31,8,11,719.00,728.36,2.00,1.43
141486,2022-01-27,Tulare,California,116213,1234.00,6107,CA,County,466195,24927.98,264.70,264.70,24927.98,9,5,3086.00,1387.57,5.00,3.00
1102280,2022-01-27,Kings,California,47349,398.00,6031,CA,County,152940,30959.20,260.23,260.23,30959.20,10,3,881.00,579.93,2.00,0.50


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,2,4.00,68.29
1,03/21/20,Imperial,4,2.21,2,0.00,68.29
2,03/22/20,Imperial,4,2.21,2,0.00,68.29
3,03/23/20,Imperial,4,2.21,2,0.00,68.29
4,03/24/20,Imperial,9,4.97,2,5.00,62.86
...,...,...,...,...,...,...,...
6805,01/23/22,Lassen,9085,29715.76,1,0.00,63.86
6806,01/24/22,Lassen,9085,29715.76,1,0.00,63.86
6807,01/25/22,Lassen,9327,30507.31,1,242.00,77.36
6808,01/26/22,Lassen,9531,31174.57,1,204.00,89.29


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6847,01/23/22,Kings,396.00,258.93,10,0.00,0.43
6848,01/24/22,Kings,397.00,259.58,10,1.00,0.57
6849,01/25/22,Kings,396.00,258.93,10,-1.00,0.50
6850,01/26/22,Kings,396.00,258.93,10,0.00,0.36


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)